In [31]:
import pickle
with open('922-Standard-SDv1.pickle', 'rb') as f:
    sdvg = pickle.load(f)
sdvg

{'0-The black chair is on the right of the wooden table': <networkx.classes.digraph.DiGraph at 0x7f325ce8fd60>,
 '1-The black chair is on the right of the wooden table': <networkx.classes.digraph.DiGraph at 0x7f325ce8ffa0>,
 '2-The black chair is on the right of the wooden table': <networkx.classes.digraph.DiGraph at 0x7f325ce90c40>,
 '3-The black chair is on the right of the wooden table': <networkx.classes.digraph.DiGraph at 0x7f325ceaf8e0>,
 '0-The round clock was mounted on the white wall': <networkx.classes.digraph.DiGraph at 0x7f325ceae950>,
 '1-The round clock was mounted on the white wall': <networkx.classes.digraph.DiGraph at 0x7f325ceae860>,
 '2-The round clock was mounted on the white wall': <networkx.classes.digraph.DiGraph at 0x7f325ceaead0>,
 '3-The round clock was mounted on the white wall': <networkx.classes.digraph.DiGraph at 0x7f325ceae4a0>,
 '0-The fluffy pillow was on the left of the striped blanket': <networkx.classes.digraph.DiGraph at 0x7f325ceaea40>,
 '1-The flu

In [32]:
import networkx as nx
class Scene_Graph:
    def __init__(self):
        self.graph = nx.MultiDiGraph()
    def add_node(self, node):
        node_idx = node
        if node in self.graph.nodes:
            for i in range(1000):
                node_idx = node + str(i)
                if node_idx not in self.graph.nodes:
                    break
                node_idx = node
            self.graph.add_node(node_idx)
            return node_idx
    def add_attribute(self, node, attribute, value):
        self.graph.nodes[node][attribute] = value
    def add_relationship(self, node1, node2, relationship):
        self.graph.add_edge(node1, node2, relationship=relationship)
    def get_nodes(self):
        return self.graph.nodes()
    def get_edges(self):
        return self.graph.edges()
    def get_attributes(self, node):
        return self.graph.nodes[node]
    def get_relationships(self, node1, node2):
        return self.graph[node1][node2][0]['relationship']

In [41]:
g1 = sdvg['2-The black chair is on the right of the wooden table']
g2 = sdvg['3-The black chair is on the right of the wooden table']

In [42]:
g1

{}

In [49]:
g1['chair']['chair1']

{'relationship': 'right$sit'}

In [9]:
g2.nodes()

NodeView(('chair', 'chair0', 'table', 'chair1', 'chair2', 'chair3', 'chair_brown', 'chair_chair', 'chair_wood', 'chair0_brown', 'chair0_chair', 'chair0_wood', 'table_brown', 'table_rectangle', 'table_wood', 'chair1_brown', 'chair1_round', 'chair1_wood', 'chair2_brown', 'chair2_round', 'chair2_wood', 'chair3_brown', 'chair3_round', 'chair3_leather'))

In [13]:
ed = nx.optimize_graph_edit_distance(g1, g2)

In [14]:
for e in ed:
    print(e)

45.0
43.0
41.0
39.0
37.0
35.0
33.0


In [15]:
nx.graph_edit_distance(g1, g2)

33.0

In [18]:
def getMCS(G_source, G_new):
    matching_graph=nx.Graph()

    for n1,n2,attr in G_new.edges(data=True):
        if G_source.has_edge(n1,n2) :
            matching_graph.add_edge(n1,n2,weight=1)

    graphs = list(nx.connected_component_subgraphs(matching_graph))

    mcs_length = 0
    mcs_graph = nx.Graph()
    for i, graph in enumerate(graphs):

        if len(graph.nodes()) > mcs_length:
            mcs_length = len(graph.nodes())
            mcs_graph = graph

    return mcs_graph

In [19]:
getMCS(g1, g2)

AttributeError: module 'networkx' has no attribute 'connected_component_subgraphs'

In [22]:
g2.nodes()

NodeView(('chair', 'chair0', 'table', 'chair1', 'chair2', 'chair3', 'chair_brown', 'chair_chair', 'chair_wood', 'chair0_brown', 'chair0_chair', 'chair0_wood', 'table_brown', 'table_rectangle', 'table_wood', 'chair1_brown', 'chair1_round', 'chair1_wood', 'chair2_brown', 'chair2_round', 'chair2_wood', 'chair3_brown', 'chair3_round', 'chair3_leather'))

In [98]:
import spacy
nlp = spacy.load('en_core_web_sm')
text = 'A dog under the cat'
doc = nlp(text)

In [97]:

# doc = nlp( "spaCy uses the terms head and child to describe the words" )
for token in doc:
    print('{0}({1}) <-- {2} -- {3}({4})'.format(token.text, token.pos_, token.dep_, token.head.text, token.head.tag_))

A(DET) <-- det -- dog(NN)
dog(NOUN) <-- nsubj -- is(VBZ)
is(AUX) <-- ROOT -- is(VBZ)
under(ADP) <-- prep -- is(VBZ)
the(DET) <-- det -- cat(NN)
cat(NOUN) <-- pobj -- under(IN)


In [ ]:
from collections import defaultdict

def parser_func(doc):
    parse_dict = defaultdict(dict)
    relation_dict = defaultdict(dict)
    for token in doc:
        if token.head.pos_ == 'NOUN' and token.dep_ == 'amod':
            if parse_dict[token.head.text].get('attributes'):
                parse_dict[token.head.text]['attributes'].append(token.text)
            else:
                parse_dict[token.head.text]['attributes'] = [token.text]
        elif token.pos_ == 'ADJ' and token.dep_ == 'conj':
            if token.head.pos_ == 'ADJ' and token.dep_ == 'amod':
                noun_node = token.head.head
                assert noun_node.pos_ == 'NOUN', 'conj error'
                parse_dict[noun_node.text]['attributes']
        elif token.pos_ == 'NOUN' and token.dep_ == 'nsubj':
            relation = token.head.text
            # next_node = token.head.head
            parse_dict[token.text]['relationship'] = relation
            relation_dict[relation]['nsubj'] = token.text
        elif token.pos_ == 'NOUN' and (token.dep_ == 'dobj' or token.dep_ == 'pobj'):
            reation = token.head.text
            subj = relation_dict[relation]
            relation_dict[subj]['obj'] = token.text
        
            # while next_node.pos_ != 'NOUN':
            #     next_node = next_node.head